# Data Cleaning

Data cleaning is the most important step of data analysis process. Most of the time data are not in the desired format. Data are often collected in raw format and it is very important to clean the data and prepare it for further analysis.

In [122]:
# import needed packages

import pandas as pd

In [123]:
# read the csv file of glassdoor job description data

df = pd.read_csv('/Users/kshit/Desktop/Python/Project Datas/Glassdoor_data.csv')

Next step is to explore the data and review all the description and summary of the data.

In [82]:
df.head()

,Column1,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [124]:
df.drop('Column1', axis=1, inplace=True)

In [125]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 14 columns):
Job Title            956 non-null object
Salary Estimate      956 non-null object
Job Description      956 non-null object
Rating               956 non-null float64
Company Name         956 non-null object
Location             956 non-null object
Headquarters         956 non-null object
Size                 956 non-null object
Founded              956 non-null int64
Type of ownership    956 non-null object
Industry             956 non-null object
Sector               956 non-null object
Revenue              956 non-null object
Competitors          956 non-null object
dtypes: float64(1), int64(1), object(12)
memory usage: 104.6+ KB


Since we are predicting the salary of Data Scientist, we will explore the elements of column Salary Estimate. We can see from the output below that there are some odd values which are needed to be cleaned and formatted.  

In [127]:
df['Salary Estimate'].value_counts()

-1                                      214
$49K-$113K (Glassdoor est.)               6
$86K-$143K (Glassdoor est.)               6
$21-$34 Per Hour(Glassdoor est.)          6
$54K-$115K (Glassdoor est.)               6
$81K-$167K (Glassdoor est.)               5
$76K-$142K (Glassdoor est.)               5
$74K-$124K (Glassdoor est.)               5
$107K-$173K (Glassdoor est.)              5
$110K-$175K (Glassdoor est.)              4
$49K-$97K (Glassdoor est.)                4
$42K-$86K (Glassdoor est.)                4
$56K-$95K (Glassdoor est.)                4
$64K-$106K (Glassdoor est.)               4
$44K-$78K (Glassdoor est.)                4
$44K-$86K (Glassdoor est.)                4
$108K-$171K (Glassdoor est.)              4
$35K-$62K (Glassdoor est.)                4
$68K-$139K (Glassdoor est.)               4
$39K-$66K (Glassdoor est.)                4
$82K-$132K (Glassdoor est.)               4
$61K-$109K (Glassdoor est.)               4
$18-$25 Per Hour(Glassdoor est.)

In [128]:
def fun(x):
    return 1 if 'per hour' in x.lower() else 0
df['Hourly']=df['Salary Estimate'].apply(fun)

In [129]:
# We will be using lambda for rest of our cleaning process even though I would not recommend it. 

df['Employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

In [130]:
df = df[df['Salary Estimate']!='-1']

In [131]:
salary = df['Salary Estimate'].str.split('(').str[0]
only_number_salary = salary.str.replace('K','').str.replace('$','')
only_number = only_number_salary.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

In [132]:
df['min_salary'] = only_number.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = only_number.apply(lambda x: int(x.split('-')[1]))
df['avg_salary'] = (df.min_salary+df.max_salary)/2

In [133]:
# Changing the wages from hourly to annual.

df['min_salary'] = df.apply(lambda x: x.min_salary*2 if x.Hourly ==1 else x.min_salary, axis =1)
df['max_salary'] = df.apply(lambda x: x.max_salary*2 if x.Hourly ==1 else x.max_salary, axis =1)

In [134]:
# Clean the company data.

df['company'] = df.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'][:-3], axis = 1)
df['company'] = df['company'].str.replace('\n','')

In [135]:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])
df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

In [136]:
df['age'] = df.Founded.apply(lambda x: x if x <1 else 2020 - x)

Data Scientist role requires high technical knowledge. We will create new columns which will give us the number of times job description mentioned the most popular skills such python programming, R programming, spark, etc.

In [137]:
df['Python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0) 
df['R_Std'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)


In [138]:
df.min_salary

0       53
1       63
2       80
3       56
4       86
5       71
6       54
7       86
8       38
9      120
10     126
11      64
12     106
13      46
14      83
15     102
16      67
17     118
18     110
19      64
20      81
21      73
22      86
23      63
25     109
26      63
27      75
28      34
29      63
30      80
      ... 
920     49
921     49
924     96
926     65
928     59
929     32
930     87
931     27
932     39
933     36
934    107
935     56
936     50
938     80
939     43
940     45
941     50
942     44
943    100
944     61
945     80
946     99
947     37
948     62
949     86
950     58
951     72
952     56
953     95
955     61
Name: min_salary, Length: 742, dtype: int64

Next we will clean the job title. It is highly probable that job title may include title other than data scientist. We will build a function to address this problem.

In [139]:
def simple(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'director' in title.lower():
        return 'director'
    elif 'manager' in title.lower():
        return 'manager'
    else:
        return 'others'

def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'jr'
    else:
        return 'others'

In [140]:
df['Job_Title']=df['Job Title'].apply(simple)

In [141]:
df['Seniority']=df['Job Title'].apply(seniority)

In [142]:
df.job_state.value_counts()

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 NC              21
 WA              21
 NJ              17
 FL              16
 OH              14
 TN              13
 DC              11
 CO              11
 UT              10
 WI              10
 IN              10
 MO               9
 AZ               9
 AL               8
 KY               6
 MI               6
 GA               6
 DE               6
 CT               5
 IA               5
 LA               4
 OR               4
 NE               4
 NM               3
 KS               3
 ID               2
 MN               2
 RI               1
 SC               1
 Los Angeles      1
Name: job_state, dtype: int64

In [143]:
# We can see that one job state is city. We will fix that and include Los Angeles in CA state. 

df['job_state']= df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'los angeles' else 'CA')
df.job_state.value_counts()

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
WA     21
NC     21
NJ     17
FL     16
OH     14
TN     13
CO     11
DC     11
WI     10
IN     10
UT     10
AZ      9
MO      9
AL      8
DE      6
MI      6
GA      6
KY      6
CT      5
IA      5
OR      4
NE      4
LA      4
NM      3
KS      3
MN      2
ID      2
RI      1
SC      1
Name: job_state, dtype: int64

In [144]:
df.describe()

,Rating,Founded,Hourly,Employer_provided,min_salary,max_salary,avg_salary,same_state,age,Python,R_Std,spark,aws,excel
count,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000,742.000000
mean,3.618868,1837.154987,0.032345,0.022911,74.719677,128.149596,100.626011,0.557951,46.591644,0.528302,0.002695,0.225067,0.237197,0.522911
std,0.801210,497.183763,0.177034,0.149721,30.980593,45.220324,38.855948,0.496965,53.778815,0.499535,0.051882,0.417908,0.425651,0.499812
min,-1.000000,-1.000000,0.000000,0.000000,15.000000,16.000000,13.500000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.300000,1939.000000,0.000000,0.000000,52.000000,96.000000,73.500000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.700000,1988.000000,0.000000,0.000000,69.500000,124.000000,97.500000,1.000000,24.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,4.000000,2007.000000,0.000000,0.000000,91.000000,155.000000,122.500000,1.000000,59.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,5.000000,2019.000000,1.000000,1.000000,202.000000,306.000000,254.000000,1.000000,276.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [145]:
# Export the cleaned data in CSV format

df.to_csv('Cleaned_data.csv', index=False)